<a href="https://colab.research.google.com/github/mzohaibnasir/GenAI/blob/main/08_openSourceLLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

popular ones:

*   Meta Llama 2
*   Google PaLM 2(behind google bard)
*   Falcon

https://github.com/eugeneyan/open-llms



# Llama 2

  # Quantization is a model compression technique. each model layer will have weights where weights are just floating point numbers. we can round floating numbers. this is quantization technique.

  ther are various quantization techniques(GGML, etc)

## Installing llama-cpp-python

In [1]:
 !nvidia-smi

Sat Apr 20 18:09:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# GPU llama-cpp-python
# Linux and Mac
! CMAKE_ARGS="-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS" pip install llama-cpp-python
! pip install huggingface_hub # to download model from hf
! pip install llama-cpp-python==0.1.78
! pip install numpy==1.23.4

In [4]:
! pip install llama-cpp-python
